IMPORT NEEDED LIBRARIES

In [18]:
##import libraries
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import seaborn as sns


LOAD AND PREPROCESS REVENUE DATA

In [20]:
# Load Revenue data 
df = pd.read_csv('Revenue_data.csv')

# Convert Order Date to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'])

# Calculate Revenue
df['Revenue'] = df['Unit Price'] * df['Quantity']

# Group monthly revenue by Revenue Stream
monthly_revenue = (
    df.groupby([pd.Grouper(key='Order Date', freq='M'), 'Revenue Stream'])['Revenue']
    .sum()
    .reset_index()
)

# Rename for Prophet
monthly_revenue.rename(columns={'Order Date': 'ds', 'Revenue': 'y'}, inplace=True)
monthly_revenue

C:\Users\SulihatSobalaje\AppData\Local\Temp\ipykernel_18832\3501586281.py:12: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df.groupby([pd.Grouper(key='Order Date', freq='M'), 'Revenue Stream'])['Revenue']


,ds,Revenue Stream,y
0,2019-01-31,License,5784.13
1,2019-01-31,Managed Service,1721.62
2,2019-01-31,Product,5828.75
3,2019-02-28,License,3821.35
4,2019-02-28,Managed Service,1814.48
...,...,...,...
223,2025-03-31,Managed Service,1008.19
224,2025-03-31,Product,5762.20
225,2025-04-30,License,4668.62
226,2025-04-30,Managed Service,1616.61


LOAD AND PREPROCESS MACROECONOMIC DATA

In [43]:
# Load and preprocess macroeconomic data
df_macro = pd.read_csv("daily_exchange_inflation_data.csv")
df_macro['ds'] = pd.to_datetime(df_macro['Order Date'])
df_macro = df_macro.drop(columns='Order Date')

# Group by month
df_macro_monthly = df_macro.groupby(pd.Grouper(key='ds', freq='M'))[['Exchange Rate', 'Inflation Rate']].mean().reset_index()
df_macro_monthly

C:\Users\SulihatSobalaje\AppData\Local\Temp\ipykernel_18832\3532420887.py:7: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_macro_monthly = df_macro.groupby(pd.Grouper(key='ds', freq='M'))[['Exchange Rate', 'Inflation Rate']].mean().reset_index()


,ds,Exchange Rate,Inflation Rate
0,2019-01-31,376.183548,11.336452
1,2019-02-28,392.585536,11.496429
2,2019-03-31,448.005484,10.462581
3,2019-04-30,441.988167,10.307833
4,2019-05-31,410.471290,12.273226
...,...,...,...
71,2024-12-31,1390.070645,23.242581
72,2025-01-31,1402.040645,24.682581
73,2025-02-28,1356.313393,24.648214
74,2025-03-31,1345.125806,24.854839


RUN FORECAST FOR EACH REVENUE STREAMS

In [42]:
import pandas as pd
from prophet import Prophet


# Loop through each revenue stream
unique_streams = monthly_revenue['Revenue Stream'].unique()

forecast_results = {}

for stream in unique_streams:
    # Filter data for the stream
    df_stream = monthly_revenue[monthly_revenue['Revenue Stream'] == stream][['ds', 'y']]
    
    # Merge with macroeconomic data
    df_model = pd.merge(df_stream, df_macro_monthly, on='ds', how='left')
    
    # Fill missing macro values if any
    df_model.fillna(method='ffill', inplace=True)

    # Build Prophet model
    model = Prophet()
    model.add_regressor('Exchange Rate')
    model.add_regressor('Inflation Rate')
    model.fit(df_model)

    # Make future dataframe
    future = model.make_future_dataframe(periods=12, freq='M')  # 12 months ahead
    future = pd.merge(future, df_macro, on='ds', how='left')
    future.fillna(method='ffill', inplace=True)

    # Forecast
    forecast = model.predict(future)
    
    # Save result
    forecast_results[stream] = forecast

    display_cols = ['ds', 'yhat', 'yhat_lower', 'yhat_upper']
    forecast_table = forecast[display_cols].tail(12).round(2)
    forecast_table['Revenue Stream'] = stream  # Add column for clarity

    display(forecast_table)


C:\Users\SulihatSobalaje\AppData\Local\Temp\ipykernel_18832\797781861.py:18: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_model.fillna(method='ffill', inplace=True)
12:13:39 - cmdstanpy - INFO - Chain [1] start processing
12:13:39 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\SulihatSobalaje\anaconda3\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
C:\Users\SulihatSobalaje\AppData\Local\Temp\ipykernel_18832\797781861.py:29: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  future.fillna(method='ffill', inplace=True)


,ds,yhat,yhat_lower,yhat_upper,Revenue Stream
76,2025-05-31,4351.49,3507.26,5228.62,License
77,2025-06-30,5315.51,4545.03,6154.99,License
78,2025-07-31,4300.29,3459.36,5105.86,License
79,2025-08-31,4384.91,3533.59,5244.14,License
80,2025-09-30,5098.09,4314.98,5925.06,License
81,2025-10-31,5232.70,4413.83,6021.57,License
82,2025-11-30,4118.45,3353.67,4903.71,License
83,2025-12-31,4748.97,3962.14,5622.66,License
84,2026-01-31,5166.91,4407.65,6037.21,License
85,2026-02-28,4531.40,3693.54,5376.33,License


C:\Users\SulihatSobalaje\AppData\Local\Temp\ipykernel_18832\797781861.py:18: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_model.fillna(method='ffill', inplace=True)
12:13:40 - cmdstanpy - INFO - Chain [1] start processing
12:13:40 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\SulihatSobalaje\anaconda3\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
C:\Users\SulihatSobalaje\AppData\Local\Temp\ipykernel_18832\797781861.py:29: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  future.fillna(method='ffill', inplace=True)


,ds,yhat,yhat_lower,yhat_upper,Revenue Stream
76,2025-05-31,1868.00,1257.88,2455.58,Managed Service
77,2025-06-30,1724.32,1166.67,2368.80,Managed Service
78,2025-07-31,1484.90,885.18,2088.28,Managed Service
79,2025-08-31,1799.20,1214.63,2383.63,Managed Service
80,2025-09-30,1518.75,908.62,2140.42,Managed Service
81,2025-10-31,1609.51,1037.87,2264.17,Managed Service
82,2025-11-30,1922.15,1325.49,2512.06,Managed Service
83,2025-12-31,1177.72,604.57,1781.19,Managed Service
84,2026-01-31,1379.32,744.08,2005.28,Managed Service
85,2026-02-28,1240.67,610.63,1888.10,Managed Service


C:\Users\SulihatSobalaje\AppData\Local\Temp\ipykernel_18832\797781861.py:18: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_model.fillna(method='ffill', inplace=True)
12:13:40 - cmdstanpy - INFO - Chain [1] start processing
12:13:41 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\SulihatSobalaje\anaconda3\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
C:\Users\SulihatSobalaje\AppData\Local\Temp\ipykernel_18832\797781861.py:29: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  future.fillna(method='ffill', inplace=True)


,ds,yhat,yhat_lower,yhat_upper,Revenue Stream
76,2025-05-31,5702.05,4620.56,6897.14,Product
77,2025-06-30,5419.65,4158.13,6593.54,Product
78,2025-07-31,5755.76,4563.51,6869.50,Product
79,2025-08-31,5859.12,4657.66,6933.41,Product
80,2025-09-30,5872.37,4672.39,6977.70,Product
81,2025-10-31,5486.07,4404.98,6580.50,Product
82,2025-11-30,5576.87,4514.02,6737.46,Product
83,2025-12-31,5343.79,4174.11,6493.94,Product
84,2026-01-31,6590.41,5495.93,7731.20,Product
85,2026-02-28,5869.30,4676.30,7003.61,Product


MODEL EVALUATION: Cross Validation and Performance Metrics

In [33]:
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

# Grouped forecast results
cv_results = {}
performance_results = {}

# Loop through each revenue stream
for stream in df['Revenue Stream'].unique():
    print(f"🔍 Running cross-validation for: {stream}")

    # Filter and prepare the dataset for the current revenue stream
    stream_df = df[df['Revenue Stream'] == stream].copy()
    stream_df['Revenue'] = stream_df['Unit Price'] * stream_df['Quantity']
    monthly = stream_df.groupby(pd.Grouper(key='Order Date', freq='M'))['Revenue'].sum().reset_index()
    monthly.columns = ['ds', 'y']

    # Automatically calculate time span
    total_months = (monthly['ds'].max().to_period('M') - monthly['ds'].min().to_period('M')).n
    total_days = total_months * 30

    # Define logic for initial, period, and horizon
    initial_days = int(total_days * 0.6)  # 60% for training
    horizon_days = int(total_days * 0.2)  # 20% for forecasting
    period_days = int(total_days * 0.2)   # 20% for step

    # Convert to strings for Prophet
    initial = f"{initial_days} days"
    period = f"{period_days} days"
    horizon = f"{horizon_days} days"

    # Fit model
    model = Prophet()
    model.fit(monthly)

    # Run cross-validation
    try:
        df_cv = cross_validation(model, initial=initial, period=period, horizon=horizon, parallel="processes")
        df_p = performance_metrics(df_cv)

        # Store results
        cv_results[stream] = df_cv
        performance_results[stream] = df_p
        print(f"✅ Done: {stream}\n")
    except Exception as e:
        print(f"❌ Failed for {stream}: {e}")
        df_cv


🔍 Running cross-validation for: License


C:\Users\SulihatSobalaje\AppData\Local\Temp\ipykernel_18832\1398524824.py:15: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = stream_df.groupby(pd.Grouper(key='Order Date', freq='M'))['Revenue'].sum().reset_index()
10:23:01 - cmdstanpy - INFO - Chain [1] start processing
10:23:02 - cmdstanpy - INFO - Chain [1] done processing


✅ Done: License

🔍 Running cross-validation for: Product


C:\Users\SulihatSobalaje\AppData\Local\Temp\ipykernel_18832\1398524824.py:15: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = stream_df.groupby(pd.Grouper(key='Order Date', freq='M'))['Revenue'].sum().reset_index()
10:23:17 - cmdstanpy - INFO - Chain [1] start processing
10:23:17 - cmdstanpy - INFO - Chain [1] done processing


✅ Done: Product

🔍 Running cross-validation for: Managed Service


C:\Users\SulihatSobalaje\AppData\Local\Temp\ipykernel_18832\1398524824.py:15: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = stream_df.groupby(pd.Grouper(key='Order Date', freq='M'))['Revenue'].sum().reset_index()
10:23:24 - cmdstanpy - INFO - Chain [1] start processing
10:23:24 - cmdstanpy - INFO - Chain [1] done processing


✅ Done: Managed Service



In [34]:
# Summary of performance metrics (MAE, RMSE, MAPE) for all streams
summary = pd.DataFrame([
    {
        'Revenue Stream': stream,
        'MAE': perf['mae'].mean(),
        'RMSE': perf['rmse'].mean(),
        'MAPE': perf['mape'].mean()
    }
    for stream, perf in performance_results.items()
]).round(2)

display(summary.sort_values(by='RMSE'))


,Revenue Stream,MAE,RMSE,MAPE
2,Managed Service,599.77,683.33,0.68
0,License,722.48,810.23,0.17
1,Product,1125.57,1243.16,0.20


In [36]:
product_groups = df.groupby('Product Name')


In [37]:
product_forecasts = []


In [39]:
final_product_forecast = pd.concat(product_forecasts, ignore_index=True)


In [47]:
from prophet import Prophet
import pandas as pd

product_forecasts = {}

unique_products = df['Product Name'].unique()

for product in unique_products:
    print(f"🔍 Running forecast for: {product}")
    
    # Filter data for one product
    product_df = df[df['Product Name'] == product]
    
    # Monthly aggregation
    monthly = product_df.groupby(pd.Grouper(key='Order Date', freq='MS'))['Revenue'].sum().reset_index()
    monthly = monthly.rename(columns={'Order Date': 'ds', 'Revenue': 'y'})
    
    # Merge with macro data
    merged_data = pd.merge(monthly, df_macro_monthly, on='ds', how='left')
    
    # Drop rows with missing values (if any)
    merged_data.dropna(inplace=True)

    # Create and fit the model
    model = Prophet()
    model.add_regressor('Exchange Rate')
    model.add_regressor('Inflation Rate')
    model.fit(merged_data)
    
    # Create future dataframe
    future = model.make_future_dataframe(periods=12, freq='MS')
    
    # Merge macro regressors into future
    future = pd.merge(future, df_macro_monthly, on='ds', how='left')
    
    # Handle any missing future regressor values (optional: fill forward)
    future.fillna(method='ffill', inplace=True)

    # Predict
    forecast = model.predict(future)

    # Save forecast
    product_forecasts[product] = forecast

    print(f"✅ Done: {product}")


🔍 Running forecast for: License 4


ValueError: Dataframe has less than 2 non-NaN rows.

In [41]:
product_to_view = 'Product 1'  # or let user select in Streamlit
display_cols = ['ds', 'Product Name', 'yhat', 'yhat_lower', 'yhat_upper']
final_product_forecast[final_product_forecast['Product Name'] == product_to_view][display_cols].tail(12).round(2)


,ds,Product Name,yhat,yhat_lower,yhat_upper
1132,2025-05-01,Product 1,645.72,300.36,979.68
1133,2025-06-01,Product 1,488.74,156.27,858.59
1134,2025-07-01,Product 1,450.92,131.69,787.61
1135,2025-08-01,Product 1,612.16,276.04,950.60
1136,2025-09-01,Product 1,388.07,58.79,729.43
1137,2025-10-01,Product 1,374.14,15.43,728.79
1138,2025-11-01,Product 1,566.47,200.91,899.22
1139,2025-12-01,Product 1,821.91,435.58,1162.75
1140,2026-01-01,Product 1,323.65,-11.25,648.30
1141,2026-02-01,Product 1,521.85,176.83,876.21
